# Mini ejemplo homografía enter dos imágenes
1) Marcar los mismos cuatro puntos y en el mismo orden en ambas imágenes
2) Moverse sobre la imagen de partida, ver resultado en el mapa del campo

In [1]:
import numpy as np
import cv2
from skimage import transform

# Captura cuatro clics del ratón (parte uno)
def get_points(event, x, y, flags, param):
    puntos = param["points"]
    img = param["image"]

    if event == cv2.EVENT_LBUTTONDOWN:  # Botón izquierdo
        puntos.append((x, y))  
        
        # Dibujar en la imagen
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], img)

        # Cuarto puntos, condición de parada
        if len(puntos) == 4:
            cv2.destroyAllWindows()
            print("Coordenadas imagen:", puntos)


# Movimiento del puntero (parte 2)
def mouse_event(event, x, y, flags, param):
    homo = param["tform"]
    img = param["image"]

    if event == cv2.EVENT_MOUSEMOVE:  
        # Transformar posición del ratón a imagen destino
        punto_A = np.array([[x, y]])  # Puntero 
        punto_B = homo(punto_A)  # Transformar a imagen destino

        # Dibuja en la imagen destino
        imgtmp = img.copy()
        cv2.circle(imgtmp, (int(punto_B[0][0]), int(punto_B[0][1])), 5, (0, 255, 0), -1)
        cv2.imshow(param["wname"], imgtmp)


############## 1. RECOPILA LOS CUATRO PUNTOS EN AMBAS IMÁGENES
#Inicializa listas depuntos
puntosA = []
puntosB = []

#Lee imágenes, crea copias de trabajo
vista = cv2.imread('capture.png')
mapa = cv2.imread('beachvolleyballcourt.png')
vistatmp = vista.copy()
mapatmp = mapa.copy()

#Vista del campo
cv2.imshow("Vista", vistatmp) 
params = {
    "points": puntosA,
    "image": vistatmp, 
    "wname": "Vista"
}
cv2.setMouseCallback("Vista", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)

#Vista ceintal
cv2.imshow("Mapa", mapa) 
params = {
    "points": puntosB,
    "image": mapatmp, 
    "wname": "Mapa"
}
cv2.setMouseCallback("Mapa", get_points, params)

# Selecciona cuatro puntos o cierra ventana
cv2.waitKey(0)
cv2.destroyAllWindows()

#Transformación de los puntos https://scikit-image.org/docs/stable/auto_examples/transform/plot_transform_types.html
tform = transform.estimate_transform('projective', np.array(puntosA), np.array(puntosB))
tf_img = transform.warp(vista, tform.inverse, output_shape=(mapa.shape))

#Muestra imagen de entrada transformada
cv2.imshow("Homografía", tf_img) 

cv2.waitKey(-1)
cv2.destroyAllWindows()

###### 2. MUESTRA POSICIÓN DEL RATÓN EN EL CAMPO, EN LA IMAGEN DEL MAPA
# Mostrar las dos imágenes
cv2.imshow("Vista", vista)
cv2.imshow("Mapa", mapa)

params = {
    "tform": tform,
    "image": mapa, 
    "wname": "Mapa"
}

# Evento de movimiento del ratón
cv2.setMouseCallback("Vista", mouse_event, params)

cv2.waitKey(0)
cv2.destroyAllWindows()


Coordenadas imagen: [(74, 450), (193, 347), (788, 341), (896, 439)]
Coordenadas imagen: [(255, 751), (255, 248), (1279, 252), (1281, 752)]


# Detectando personas y esquinas en una imagen de campo de fútbol

In [2]:
import cv2
import numpy as np

from ultralytics import YOLO  #Necesario para detectar con YOLO

#Intersección enter dos líneas
def line_intersection(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    # Calcular  pendientes e intersecciones
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:  # Líneas paralelas o coincidentes
        return None

    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom

    return (intersect_x, intersect_y)

#Comprueba que el punto esté dentro de la imagen
def is_within_image(point, image_width, image_height):
    x, y = point
    return 0 <= x < image_width and 0 <= y < image_height

#Carga modelo YOLO
model = YOLO('yolo11n.pt') #Contenedores
# Etiqueta de las clases o clase de interés
classNames = ["person"]

# Carga imagen
image = cv2.imread("capture.png")
height, width, _ = image.shape  
# Convierte a HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
# Muestra entrada
cv2.imshow("Input", image)

# Rango de verde de interés en HSV
lower_sand = np.array([10, 10, 80])
upper_sand = np.array([40, 160, 255])

# Máscara de zona arena
mask = cv2.inRange(hsv, lower_sand, upper_sand)
# Muestra máscara
cv2.imshow("Máscara", mask)

# Contornos en la máscara
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Imagen vacía
final_mask = np.zeros_like(mask)

# Filtrar contornos con un área mayor a 500 píxeles
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 500:
        # Rellenar los contornos seleccionados
        cv2.drawContours(final_mask, [contour], -1, 255, thickness=cv2.FILLED)

#Muestra máscara resultado
cv2.imshow("MáscaraF", final_mask)

# Aplicar máscara sobre la imagen original
masked_image = cv2.bitwise_and(image, image, mask=final_mask)

#Aplica YOLO
results = model(masked_image) 
# Para cada detección la muestra
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(masked_image, (x1,y1), (x2,y2), (255, 255, 255), 2)

cv2.imshow("CampoF", masked_image)


# Convierte a grises y detecta de bordes
gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)
#Muestra bordes
cv2.imshow("edges", edges)

# Detecta líneas con la transformada de Hough
min_line_length = gray.shape[0] // 3  # Longitud mínima de la linea
lineas_campo = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=30, minLineLength=min_line_length, maxLineGap=10)

# Filtrar las nlineas líneas más próximas a la parte inferior de la imagen
nlineas = 10
if lineas_campo is not None:
    lineas_campo = sorted(lineas_campo, key=lambda l: min(l[0][1], l[0][3]), reverse=True)
    lineas_campo = lineas_campo[:10]

# Inicializa grupos de líneas y umbral de orientación para separar
group_1 = []  
group_2 = []  
angle_threshold = 10

# Agrupa líneas por orientación
if lineas_campo is not None:
    for line in lineas_campo:
        x1, y1, x2, y2 = line[0]
        
        # Evita división por cero en líneas verticales
        if x2 - x1 == 0:
            angle = 90  
        else:
            # Ángulo en grados
            angle = np.degrees(np.arctan((y2 - y1) / (x2 - x1)))
            angle = abs(angle) 
        
        # Asignar grupo
        if abs(angle) < angle_threshold:  # Cercano a horizontal
            group_1.append(line)
        else:  # Resto
            group_2.append(line)

# Dibuja cada grupo con un color
if group_1:
    for line in group_1:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Azul para grupo 1

if group_2:
    for line in group_2:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para grupo 2

#INTERSECCIONES
# Obtener las intersecciones entre líneas de grupo diferente
image_height, image_width = image.shape[:2]
intersections = []

for i in range(len(group_1)):
    for j in range(len(group_2)):
        line1 = group_1[i][0]
        line2 = group_2[j][0]

        intersection = line_intersection(line1, line2)
        # Chequea que esté en la imagen
        if intersection and is_within_image(intersection, image_width, image_height):
            intersections.append(intersection)

# Dibuja  intersecciones en la imagen
for point in intersections:
    x, y = point
    cv2.circle(image, (int(x), int(y)), 5, (255, 0, 255), -1)

# Muestra imagen con líneas e intersecciones
cv2.imshow("Líneas y esquinas detectadas", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 352x640 2 persons, 968.8ms
Speed: 86.0ms preprocess, 968.8ms inference, 170.6ms postprocess per image at shape (1, 3, 352, 640)
